# Audio Synthesis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
SAMPLE_RATE = 44_100  # See https://en.wikipedia.org/wiki/Sampling_(signal_processing)

# Create a signal

In [ ]:
duration = 1
frequency = 440
t = np.linspace(0, duration, num=int(duration * SAMPLE_RATE))
signal = np.sin(frequency * t * 2 * np.pi)


# Plot the signal

In [ ]:
single_cycle_end = int(
    1/frequency * len(signal)
)

single_cycle = signal[:single_cycle_end]

plt.plot(single_cycle);

In [ ]:
# Plot the whole signal. Note that it will not be very clear because of the lack of resolution
plt.plot(signal);

# Play the signal

In [ ]:
# Helper function so we can play np.ndarray as audio
def convert_to_audio(y):
    y *= 32767 / np.max(np.abs(y))
    y = y.astype(np.int16)
    return y
    

In [ ]:
audio = convert_to_audio(signal)
ipd.Audio(audio, rate=SAMPLE_RATE)

# Create an amplifier

### Create the attack envelope

In [ ]:
# First, create the attack envelope. 
# Increase the signal from over a given duration
attack = ...  # in seconds
attack_amp = np.linspace(...)

In [ ]:
# Combine the amplifier and the original signal
amped_signal = ...

plt.plot(amped_signal);

### Implement Decay, Sustain and Release

In [ ]:
# Now, do the same thing for Decay, Sustain and Release
# Create one single envelope and combine it with the original signal

### Combine all envelopes into one

In [ ]:
amp = np.concatenate([...])

### Amplify the original signal

In [ ]:
amped_signal = ...

### Trim the audio (removing zeros at start and end)

In [ ]:

amped_signal = np.trim_zeros(amped_signal)

# Plot the amplified signal

In [ ]:
plt.plot(amped_signal);

# Play the amplified audio

In [ ]:
audio = convert_to_audio(amped_signal)
ipd.Audio(audio, rate=SAMPLE_RATE)

# Filtering the audio

In [ ]:
from scipy.signal import butter, filtfilt

def butter_filter(data: np.ndarray, cutoff: float, order: int = 2, filter_type: str = 'lowpass') -> np.ndarray:
    nyquist_frequency = 0.5 * SAMPLE_RATE
    normal_cutoff = cutoff / nyquist_frequency

    # Get the filter coefficients
    b, a = butter(order, normal_cutoff, btype=filter_type, analog=False)

    # apply the filter to the signal
    y = filtfilt(b, a, data)
    return y

In [ ]:
filter_frequency = 1000
filtered_signal = butter_filter(amped_signal, frequency=filter_frequency)
plt.plot(filtered_signal);

In [ ]:
audio = convert_to_audio(filtered_signal)
ipd.Audio(audio, rate=SAMPLE_RATE)